In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import tensorflow as tf 
import numpy as np 
import pandas as pd 
import nltk
import time
import math
import random
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score 
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np 
import matplotlib.pyplot as plt 
np.random.seed(0)

In [ ]:
from skipthought import SkipthoughtModel
from skipthought.data_utils import TextData

In [ ]:
model = SkipthoughtModel(cell_type='gru', num_hidden=20, num_layers=2, embedding_size=20, max_vocab_size=500,
                 learning_rate=0.01, decay_rate=0.1, decay_steps=100, grad_clip=0.001, num_samples=500,\
                         max_length_decoder=100)

In [ ]:
textdata = TextData("../../data/train-neg.txt")
lines = textdata.dataset

In [ ]:
batch_size=32
num_epochs=10
verbose=100
save_every=1000
save_dir='save/'

In [ ]:
triples = textdata.make_triples(lines)

In [ ]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    saver = tf.train.Saver(tf.all_variables(), max_to_keep=20)

    num_batches = len(triples[0])//batch_size
    loss_history = []
    for e in range(num_epochs):
        it = textdata.triples_data_iterator(triples[0], triples[1], triples[2],
                                                textdata.max_len, batch_size, shuffle=True)
        for b, batch in enumerate(it):
            train_op, loss, feed_dict = model.train_step(*batch)

            start_time = time.time()
            batch_loss, _ = sess.run([loss, train_op], feed_dict=feed_dict)
            batch_perplexity = math.exp(float(batch_loss)) if batch_loss < 300 else float("inf")
            end_time = time.time()

            loss_history.append(batch_loss)
            if b % verbose == 0:
                print("{}/{} (epoch {}), train_loss = {:.3f}, perplexity = {:.3f}, time/batch = {:.3f}" \
                        .format(e * num_batches + b,
                        num_epochs * num_batches,
                        e, batch_loss, batch_perplexity, end_time - start_time))